<div align="center">
  <h1><b> RL Theory </b></h1>
</div>
<br>

<b>Author: <a target="_blank" href="https://github.com/camponogaraviera">Lucas Camponogara Viera</a>.

In [1]:
%autosave 30

Autosaving every 30 seconds


In [3]:
#!jupyter nbconvert --to pdf supplementary.ipynb

---
There is an assortment of extensive resources out there, and this notebook has no aspiration to be a surrogate. The recommended textbook for reinforcement learning is c.f. Ref. \[[1](#2)], and for deep learning is c.f. Ref. \[[2](#2)].

---

# Table of Contents

- **[Math Notation](#math)**
- **[The RL Framework/Problem](#rl)**
- **[Glossary](#gloss)**
    - Learning frameworks
    - Markov Decision Process (MDP)
    - Markov Chain
    - Agent, Environment and Model
    - Agent types
    - Environment types
    - State space and Observation
    - Action space
    - State transition probability distribution
    - History/Trajectory/Episode/Rollout
    - Horizon and Experience
    - Reward signal, Reward function and Return
    - Policy
    - Value Functions
    - Bellman Equations
    - Advantage Function
    - Exploitation vs Exploration: the RL tradeoff
    - $\epsilon$-greedy policy
    - Importance sampling
    - Bootstrapping
    - Deadly triad issue        
- **[Taxonomy (extended)](#tax)**
    - Model-based RL
    - Model-free RL
    - Dynamic programming methods
        - Policy Iteration
        - Value Iteration
            - Q-learning
        - Modified Policy Iteration
    - Policy optimization methods
        - Derivative-free Optimization
        - Policy Gradient
    - Deep RL
        - Deep Q-Network (DQN)
        - Actor-Critic Methods
- **[Open Challenges in RL](#open)**
    - Sparse rewards
    - Alignment problem
- **[Avoiding memory bottlenecks in deep neural networks](#memory)**
- **[Policy Optimization Algorithms](#algo)**
    - Derivative-free Optimization Algorithms
        - Cross Entropy Method (CEM)
    - Policy Gradient Algorithms
    - Actor-Critic Algorithms
- **[References](#References)**

# 1. Setup &nbsp; <a href="#"><img valign="middle" height="45px" src="https://img.icons8.com/jupyter" width="45" hspace="0px" vspace="0px"></a>

🪨 LaTeX config.

[//]: # (Math declarations:)

$\DeclareMathOperator{\Tr}{Tr}$
$\DeclareMathOperator{\HS}{HS}$
$\DeclareMathOperator{\T}{T}$

In [6]:
%%javascript MathJax.Hub.Config({ TeX: { equationNumbers: { autoNumber:"AMS" } } });
// JavaScript extension for automatic equation numbering.

<IPython.core.display.Javascript object>

# 2. Math notation<a name="math" />

| Meaning                          | LaTeX                | symbol
| :---:                            | :---:                | :---: 
| Is equal to                      | `=`                  | $=$ 
| Is not equal to                  | `\ne`                | $\ne$ 
| Is identical/congruent to        | `\equiv`             | $\equiv$ 
| Is defined as                    | `:=`                 | $:=$ 
| Is dot equivalent to             | `\doteq`             | $\doteq$
| Is approximately equal to        | `\approx`            | $\approx$ 
| is similar to                    | `\sim`               | $\sim$ 
| Is proportional to               | `\propto`            | $\propto$
| Is less than or equal to         | `\le`                | $\le$
| Is greater than or equal to      | `geq`                | $\ge$
| Therefore                        | `\therefore `        | $\therefore$ 
| $p$ is equivalent to/implies $q$ | `p\Leftrightarrow q`    | $p\Leftrightarrow q$ 
| Plus or minus                    | `\pm`                | $\pm$
| Is an element of                 | `\in`                | $\in$
| Is not an element of             | `\notin`             | $\notin$
| Is a subset of                   | `\subseteq`          | $\subseteq$
| Is a proper subset of            | `\subset`            | $\subset$
| Set intersection                 | `\cap`               | $\cap$
| Set Union                        | `\cup`               | $\cup$
| The set of all $x$ such that     | `\{x: \cdots\}`      | $\{x: \cdots\}$
| The empty set                    | `\varnothing`        | $\varnothing$
| The null element                 | `\mathbb{O}`         | $\mathbb{O}$
| Exponential                      | `\exp`               | $\exp$
| Product over $j$                 | `\prod_{j}`          | $\prod_{j}$
| Summation over $j$               | `\sum_{j}`           | $\sum_{j}$
| Limit as $N$ approaches to $a$   | `lim_{N\rightarrow a}` | $lim_{N\rightarrow a}$
| Random variable                  | `x_j`                |$x_j$ 
| Number of outcomes of $x_j$      | `N_j`                |$N_j$ 
| Number of measurements           | `N`                  |$N$ 
| Probability of $x_j$           | `\Pr(x_{j})`         | $Pr(x_{j})$
| Probability distribution         | `\{p_j\}`            | $\{p_j\}$
| Mean value (average)             | `\bar{x}`            | $\bar{x}$
| Expected value                   | `\langle x \rangle`  | $\langle x \rangle$
| Conjugate/Hermitian transpose    | `\dagger`            | $\dagger$

# 3. The RL Framework/Problem <a name="rl" />

Main reference for this section: [John Schulman's PhD Thesis](https://www2.eecs.berkeley.edu/Pubs/TechRpts/2016/EECS-2016-217.html), and [David Silver's RL course](https://www.davidsilver.uk/teaching/), and [Open AI](https://spinningup.openai.com/en/latest/spinningup/rl_intro.html).

![MDP](../assets/mdp.png)

Image showing an Agent-Environment feedback loop in the framework of a Markov Decision Process (MDP).

© Image Credits: [Open AI](https://spinningup.openai.com/en/latest/spinningup/rl_intro.html).

Reinforcement learning (RL) is a general-purpose framework for artificial intelligence and a paradigm of machine learning concerned with making a sequence of optimal actions (a.k.a control or decision making) through a trial-and-error interaction (learning process) between an agent and a previously unknown environment. This interaction is formally described in the framework of `Markov Decision Processes (MDP)`.

There are two fundamental problem setups (frameworks) in tabular rasa sequential decision making.

- Model-free RL a.k.a The Reinforcement Learning setup (framework):
  - The environment is initially unknown.
  - The agent learns solely via interaction with the environment.
  - The agent improves its Policy function.
  - There is no transition probability distribution (function) giving the conditional probability of the next state and associated reward signal given the current state-action pair.
  

- Model-based RL a.k.a The Planning setup (framework):
  - A model of the environment is known (the agent knows all the rules of the game, the differential equations of motion, etc.).
  - The agent performs computations with the model (without any external interaction).
  - The agent improves its Policy function. Known as deliberation, reasoning, introspection, pondering, thought, and search.
  - There is a transition probability distribution (function) giving the conditional probability of the next state and associated reward signal given the current state-action pair.

In the RL setting, the environment gives a state $s_t$ and a reward signal $r_t$ to the agent who performs an action $a_t$ at time step $t$ according to some Policy function $\pi(\theta)$. This process repeats in a feedback loop.

- At each time step, the agent:
  - Executes action $a_t$.
  - Receives observation $O_t = s_t$.
  - Receives scalar feedback reward signal $r_t$.


- The environment:
  - Receives action $a_t$.
  - Emits observation $O_{t+1} = s_{t+1}$.
  - Emits scalar reward signal $r_{t+1}$.

## 3.1 RL Goal

**Reward Hypothesis:** `all goals can be described by the maximization of the expected cumulative reward, regardless of the return function (infinite-horizon discounted, or finite-horizon undiscounted) or the Policy (deterministic, or stochastic).`

**The agent's goal (a.k.a objective) in RL** is to select a parameterized policy $\pi_{\theta}$ that maximizes the expected return a.k.a expected cumulative reward $J(\pi_{\theta}) = E[R(τ)]$ under said policy over an episode (a.k.a trajectory or rollout). This is achieved by optimizing the parameters $\theta$ of the parameterized policy $\pi_{\theta}$ (e.g., a neural network). The optimization typically involves computing the gradient of the expected return $\nabla_\theta J(\pi_\theta)$ and using it to improve the policy. The expected return $J(\pi_{\theta})$ is also known as performance objective.

The goal can be represented by the following equation:

\begin{eqnarray}
\text{max } \eta(\pi)&=&\text{max } J(\pi_{\theta}) \\
&=& \text{max } \mathbb{E}[\mathcal{R}(\tau) | \pi_{\theta}] \\
&=&\text{max } \mathbb{E}\left[\sum_{t'=t}^{T \rightarrow \infty} \gamma^{t'-t} r_{t'} | \pi_{\theta}\right], t'-t \rightarrow n \\
&=&\text{max } \mathbb{E}\left[\sum_{n=0}^{T \rightarrow \infty} \gamma^n r_{t+n} | \pi_{\theta}\right] \\
&=& \text{max } \mathbb{E}[r_t + \gamma r_{t+1} + \gamma^2 r_{t+2} + \cdots + \gamma^{T} r_{t+T} | \pi_{\theta}] \in {\rm I\!R}.
\end{eqnarray}

This can be restated in terms of the optimal Policy function $\pi^*$ as:
    
$$\pi^* = \underset{\pi}{\operatorname{arg max}} J(\pi).$$

The expected return considering a finite-horizon undiscounted return is given by:

$$J(\pi_{\theta}) = \mathbb{E} [\mathcal{R}(\tau)] = \sum_{r \in \mathcal{R}} r \sum_{s \in s'} \rho(s', r | s, a).$$

The expected return considering an infinite-horizon discounted return is given by:

$$J(\pi_{\theta}) = \mathbb{E}_{\tau \sim \pi}[\mathcal{R}(\tau)] = \int_{\tau} \underbrace{\mathbb{P}(\tau | \pi)}_{\text{Trajectory prob.}}  \underbrace{\mathcal{R}(\tau)}_{\text{Return}}.$$

# 4. Glossary<a name="gloss" />

Main reference for this section: [Sutton & Barto c.f. [4]](#), [David Silver's RL course](https://www.davidsilver.uk/teaching/), and [Open AI](https://spinningup.openai.com/en/latest/spinningup/rl_intro.html).

## 4.1 Learning frameworks

- **Online RL:** the algorithm uses only data from trajectories collected during runtime via direct interaction with the environment.


- **Offline RL:** the algorithm uses an external dataset of logged transitions collected only once using some external behavioral Policy that is not assumed to be Markov. This dataset can be hand crafted by the engineer.


- **On-Policy RL:** each update/optimization of the Policy parameters uses data collected only with the latest (most recent) Policy. Each Experience (a.k.a Transition) is used only once before it is discarded. The target Policy (the one being evaluated and improved) that the agent learns is equal to the behavior Policy (the one used for action selection). Examples of algorithms: Policy iteration, Policy evaluation, SARSA, and Policy optimization methods (DFO, Policy Gradient: PPO, A2C, etc.).


- **Off-Policy RL:** all the data collected by the model's latest Policy is accumulated into a replay buffer for later use. The target Policy is different from the behavior Policy. This method is used to increase sample efficiency and also for: continuous exploration, learning from demonstration and parallel learning. Examples of algorithms: Q-learning, DQN, DDPG, expected SARSA. 
    - **Target Policy $\pi$:** is the Policy being evaluated and improved during off-policy learning, i.e, is the Policy that the agent learns.

    - **Behaviour Policy $b$:** is the agent's behavioral function used for action selection during the exploration phase of the off-policy learning, where the agent explores the environment to gather experiences. 

## 4.2 Markov Decision Process (MDP)

- **Markov Decision Process (MDP):** is a `discrete-time stochastic control process` that models the interaction between an agent with a stochastic environment. The process obeys the `Markov Property` and is defined as a 5-tuple:

$$(S, A, R, \mathcal{P}, \rho_0).$$

- $S \in {\rm I\!R}^{d}$ is the state space.
- $A \in {\rm I\!R}^{d}$ is the action space.
- $R: S \times A \times S \rightarrow {\rm I\!R}$ is the reward function.
- $\mathcal{P}: S \times A \rightarrow  S \in {\rm I\!R}^{d}$ is the transition probability distribution.
- $\rho_0 \in S$ is the initial (starting) state distribution.


- **Markov Property:** a state $s_t$ is a Markov state (information state) if and only if the probability of the next state $s_{t+1}$ conditioned to the current state $s_t$ is equal to the probability of the next state conditioned to the full history. Transitions do not depend on prior history, only on the current state and action. In other words: the future is independent of the past given the present. 

$$\mathbb{P}[s_{t+1} | s_t] = \mathbb{P}[s_{t+1} | s_1, \cdots, s_T].$$ 
Also appearing as:
$$\mathbb{P}(r, s | s_t, a_t) = \mathbb{P}(r, s | \mathcal{H}_t, a_t).$$

- **State space ($S$):** the set of all possible states of the environment.


- **Action space ($A$):** the set of all possible actions the agent can select at each time step.
    - **Discrete action space:** allows only a `limited number of actions` (Atari, Go, Chess, left or right in cart-pole env.).
    - **Continuous action space:** allows a `continuous range of actions` (turning the wheel of a self-driving car). Here, actions are encoded as real-valued vectors.


- **Reward function ($R$):** encodes the reward signal $r_t$: $$r_{t} = R(s_t, a_t, s_{t+1}) \in {\rm I\!R}.$$


- **Transition probability distribution ($\mathcal{P}$):** a function that gives the conditional probability of ending up in state $s_{t+1}$ with reward signal $r_{t+1}$ after taking action $a_t$ in state $s_t$. It can be encoded as a matrix of real numbers. Every model-based RL framework has a transition probability function, while model-free RL does not.

$$\mathcal{P}(S) = \mathbb{P}(r_{t+1}, s_{t+1} | s_t, a_t) \in {\rm I\!R}^{d}.$$


- **Initial (start)-state distribution ($\rho_0$):** the initial state $s_0$ is sampled from a probability distribution $\rho_0$.

$$s_0 \sim \rho_0(\cdot).$$ 

The squiggle sign $\sim$ (tilde) means `"sampled from"`.

## 4.3 Markov Chain

A markov chain share the markov property with the markov decision process, however, it differs in the fact that it is a $2$-tuple 

$$(S, \mathcal{P})$$ 

containing only the state and transition probability distribution. 

## 4.4 Agent, Environment and Model

- **Agent:** any mechanism or thing (robot, actuator, sensor, etc.) that can perform an action in an environment given the current state or observation.


- **Environment:** is the agent's world. At every time step the env. returns an observation (environment state) and a reward signal.


- **Model:** the agent's representation of the environment. The model predicts what the environment will do next. 
    - **State transition model ($\mathcal{P}$):** predicts the next state, i.e, the environment dynamics. It gives the probability of being in the next state provided the previous state and action. The agent can learn the transition probability distribution:   
    
      $$\mathcal{P}_{ss'}^a = \mathbb{P}\Big[S' = s' = s_{t+1} | S=s_t, A=a_t\Big] \in {\rm I\!R}^{d}.$$

    - **Reward model ($\mathcal{R}$):** predicts the next (immediate) reward signal. Gives the expected reward given the current state and action. 
    
    $$\mathcal{R}_s^a = \mathbb{E}\Big[R (\tau) = G_t | S = s_t, A = a_t\Big] \in {\rm I\!R}.$$

## 4.5 Agent types

- **Value based:** stores/learn the Action-Value function (Q-value). The Policy function is implicit.
- **Policy based:** stores/learn the Policy function directly. The user can either define a deterministic Policy, or the Policy can be learned in a deep RL framework via some neural network. There's no Action-Value function (Q-value) here!
- **Actor-Critic:** the agent stores/learns both the Policy function and the Action-Value function (Q-value).

## 4.6 Environment types

- **Fully observable env.:** all states of the environment are accessible to the agent. 

$$O_t = S^e_t = S^a_t.$$ 

Meaning: the observation that the agent sees is equal to the environment state and the agent state. Formally, this is a `Markov decision process (MDP)`.

- **Partially observable env.:** only some states of the env. are accessible to the agent who indirectly observes environment. Formally, this is a `partially observable Markov decision process (POMDP)`.
    - Agent must construct its own state representation $S_t^a$.
      - 1st approach or naive approach considers the agent state as the whole history: $S_t^a = \mathcal{H}_t$.
      - 2nd approach uses Beliefs of environment state (bayesian  approach), i.e, build a probability distribution for the agent state: 

      $$S_t^a = [\mathbb{P}(S_t^e = s^1), \cdots, \mathbb{P}(S_t^e = s^n)].$$ 

      Where $S_t^a$ is a vector of probabilities of finding the environment state in the state $s^1$ to $s^n$.
      - 3rd approach uses a recurrent neural network: 

      $$S_t^a = \sigma(S_{t-1}^a W_s + O_t W_o).$$

- **Deterministic env.:** the next `state is completely determined` by the current state and action. 
    - **Process is static:** the environment remains `stationary during runtime` and only changes upon actions. $$s_{t+1} = f(s_t, a_t) \in {\rm I\!R}^{d_s} \text{ (For a deterministic environment)}.$$

- **Stochastic env.:** the next `state is randomly sampled` from a probability distribution.
    - **Process is dynamic:** `state and reward signal changes` on its own during runtime. $$s_{t+1} \sim \mathbb{P}(r_{t+1}, s_{t+1} | s_t, a_t) \in {\rm I\!R}^{d_s} \text{ (For a stochastic environment)}.$$
    
- **Episodic:** the agent's experience is divided into periods or `time steps`.

- **Single or multiple agent.**




## 4.7 State space and observation

- **State space ($S$):** the set of all possible states of the environment.


- **Observation ($O_t$):** a characteristic of the environment not necessarily accessible to the agent.  

    - **Environment State ($O_t = S_t^e \in S$):** the private representation of the environment. The env. can be at only one state at a given time step.
    
    - **Agent State ($O_t = S_t^a = s_t \in S$):** a function of the history $\mathcal{H}_t$ used to train/learn the Policy.

The state can be deterministic, i.e., the next state is completely determined by the current state and action:

$$s_{t+1} = f(s_t, a_t) \in {\rm I\!R}^{d_s} \text{ (For a deterministic environment)}.$$

The state can be stochastic, i.e., the next state is randomly sampled from a probability distribution:

$$s_{t+1} \sim \mathbb{P}(r_{t+1}, s_{t+1} | s_t, a_t) \in {\rm I\!R}^{d_s} \text{ (For a stochastic environment)}.$$

Where $\mathbb{P}(r_{t+1}, s_{t+1} | s_t, a_t)$ is the state transition probability distribution.

## 4.8 Action space

- **Action space ($A$):** the set of all possible actions the agent can select at each time step.
    - **Discrete action space:** allows only a `limited number of actions` (Atari, Go, Chess).
    - **Continuous action space:** allows a `continuous range of actions` (self-driving car). Here, actions are encoded as real-valued vectors.
    
    
- **Action ($a_t \in A$):** The agent's decision making in the env. The environment goes from state $s_t$ to state $s_{t+1}$ under action $a_t$. The agent can `act randomly` or `act greedily`.

For either discrete or continuous action space, actions are obtained from the Policy function according to:

$$a_t = \pi(s_t) \equiv \mu(s_t) \in {\rm I\!R}^{d_a} \text{ (Action for a deterministic policy)}.$$
$$a_t \sim \pi(\cdot | s_t) = \mathbb{P}[A_t = a_t | S_t = s_t] \in {\rm I\!R}^{d_a} \text{ (Action sample for a stochastic policy)}.$$

And for parameterized policies:

$$a_t = \pi_{\theta}(s_t) \equiv \mu_{\theta}(s_t) \in {\rm I\!R}^{d_a} \text{ (Action for a parameterized deterministic policy)}.$$
$$a_t \sim \pi_{\theta}(\cdot | s_t) = \mathbb{P}[A_t = a_t | S_t = s_t] \in {\rm I\!R}^{d_a} \text{ (Action sample for a parameterized stochastic policy)}.$$

When the Policy function is implicit, i.e, when the agent does not have direct access to the Policy, actions are computed from the $Q$-value according to:

$$a(s_t)=\underset{a}{\operatorname{arg\,max}} \ Q(s_t, a_t) \text{ (Discrete Action space)}.$$

$$a(s_t) \approx \text{arg} \ Q(s_t, \mu(s_t)) \text{ (Continuous Action space)}.$$

In this case, the Policy is deterministic.

## 4.9 State transition probability distribution

Is a function that gives the conditional probability of ending up in state $s_{t+1}$ with reward signal $r_{t+1}$ after taking action $a_t$ in state $s_t$. It can be encoded as a matrix of real numbers. Every model-based RL framework has a transition probability function, while model-free RL does not.

$$\mathcal{P}(S) = \mathbb{P}(r_{t+1}, s_{t+1} | s_t, a_t) \in {\rm I\!R}^{d}.$$

## 4.10 History/Trajectory/Episode/Rollout

- **History ($\mathcal{H}_t$) a.k.a Trajectory {$\tau_i$} Episode or Rollout:** is a sequence of time steps (**transitions or experiences**) from an initial state $s_0$ to a terminal state $s_T$ of an episodic environment. Each time step contains one observation (state) $s_t$, one action $a_t$, and one reward signal $r_t$. **Think of it as an entire gameplay**.

$$\mathcal{H}_t \doteq \mathcal{D}_k=\{\tau_i\} = (s_0, a_0, r_1, s_1, \cdots , s_{T-1}, a_{T-1}, r_T, s_T) \in Tuple.$$

## 4.11 Horizon and Experience

- **Horizon ($h$):** the terminal state of an episode. In a finite horizon, there can be a `non-stationary optimal policy`. In an infinite horizon, there can be a `stationary optimal policy`.


- **Experience a.k.a Transition:** a tuple consisting of the current state $s_t$, current action $a_t$, current reward signal $r_t$, next state $s_{t+1}$ and an optional done (boolean) value $d_t$.


- **Experience replay buffer:** a memory card of collected past experiences. Used in off-policy algorithms such as Q-learning, DDPG, D4PG, MADDPG, TD3, SAC, and ACER.
    
    $$(s_t, a_t, r_{t}, s_{t+1}, d_{t}), (s_{t+1}, a_{t+1}, r_{t+1}, s_{t+2}, d_{t+1}), \cdots$$

## 4.12 Reward signal, Reward function and Return

- **Reward function ($R$):**  encodes the reward signal $$r_{t} = R(s_t, a_t, s_{t+1}) \in {\rm I\!R}.$$


- **Reward signal ($r_t$):** a scalar feedback from the environment, a measure of how good the current state is after taking action $a_{t-1}$. It defines the goal of the RL problem. It drives the agent's action in the future, but do not influence how the agent decides current action. It is encoded as a `single scalar value` (positive or negative) or as a `function of state and action`.

$$r_t \in {\rm I\!R}.$$
$$r_{t} = R(s_t, a_t, s_{t+1}) \in {\rm I\!R}.$$


- **Return ($G_t$):** the total (comulative) sum of rewards over one episode (or trajectory) starting from state $s_t$ onwards. The agent's goal is to maximize the expected return or cumulative reward.

    - **Finite-horizon undiscounted return:** $$\mathcal{R}(\tau) \doteq G_t = \sum_{t=0}^T r_t \in {\rm I\!R}.$$
    
    - **Infinite-horizon discounted return:** $$\mathcal{R}(\tau) \doteq G_t = \sum_{n=0}^{\infty} \gamma^n r_{t+n+1} \in {\rm I\!R} = r_{t+1} + \gamma G_{t+1}. \text{ Using Recursion.}$$ Where $\gamma$ is known as discount factor.


- **Discount factor ($\gamma$):** is a scalar parameter used to guarantee the convergence of the infinite series above.

$$\gamma \in [0, 1] \in {\rm I\!R}, \text{where } 1 \text{ means no discount}.$$

## 4.13 Performance objective

**Note: the following definition, using the integral sign $\int (\cdot)$, consider the infinite-horizon discounted return function. For the finite-horizon undiscounted return, one should replace the integral sign by the summation sign $\sum (\cdot)$.**

Every model-based RL framework has a transition probability function, and every trajectory has an associate probability and cumulative reward (return). **If both the environment transitions and the Policy function are stochastic** (given by a probability distribution), the probability of a trajectory with $T$ time steps under Policy $\pi$ is given by: 

$$\mathbb{P}(\tau | \pi) = \underbrace{\rho_0(s_0)}_{\text{start-state dist.}} \prod_{t=0}^{T-1} \underbrace{\underbrace{\mathbb{P}(r_{t+1}, s_{t+1}|s_t,a_t)}_{\text{State transition prob.}}}_{\text{Env. model.}} \cdot \underbrace{\underbrace{\pi(a_t | s_t)}_{\text{Action prob.}}}_{\text{Control function.}}.$$

In the framework of model-based RL, where a state transition probability distribution is known, one can then write the expected return (for whichever measure) as:

$$J(\pi) = \mathbb{E}_{\tau \sim \pi}[\mathcal{R}(\tau)] = \int_{\tau} \underbrace{\mathbb{P}(\tau | \pi)}_{\text{Trajectory prob.}}  \underbrace{\mathcal{R}(\tau)}_{\text{Return}}.$$ 

Where:

- $\tau = (s_0, a_0, s_1, a_1, \dots)$ represents a trajectory (a sequence of states and actions).
- $\pi(a_t, s_t)$ denotes the Policy function, i.e, the stochastic action probability distribution a.k.a control function.
- $\mathbb{P}(r_{t+1}, s_{t+1}|s_t,a_t)$ denotes the stochastic state transition probability distribution a.k.a the environment model.
- $\mathbb{P(\tau | \pi)}$: is the probability of getting a trajectory $\tau$ with $T$ time steps acting according to policy $\pi$.
- $\rho_0$ denotes the initial (start)-state probability distribution. The initial state $s_0$ is sampled from the probability distribution: $s_0 \sim \rho_0(\cdot).$
- $\mathcal{R}(\tau) = \sum_{t=0}^{T} \gamma^t r_t$ is the cumulative (discounted) reward for the trajectory over an episode.
- $\gamma \in [0, 1]$ is the discount factor.

When a neural network (NN) is used as the expressive nonlinear function approximator to estimate/represent a parameterized Policy function $\pi_{\theta}$ (the parameters $\theta$ are the weights of the NN), the equation above can be rewritten as:

$$J(\pi_{\theta}) =  \mathbb{E}_{\tau \sim \pi_{\theta}}[\mathcal{R}(\tau)] = \int_{\tau} \underbrace{\mathbb{P}(\tau | \pi_{\theta})}_{\text{Trajectory prob.}}  \underbrace{\mathcal{R}(\tau)}_{\text{Return}}.$$

Where:

- $\pi_{\theta}$ is the parameterized policy represented by a neural network.

## 4.14 Policy

- **Policy $\pi$:** is a function that maps one state to an action and gives the action probabilities. The agent's goal is to find the optimal Policy that drives actions to the highest reward. The Policy can be encoded as a `simple function`, a `lookup table` (tabular methods), or as a parameterized neural network $\pi_{\theta}$. The mapping is denoted by: 

$$ \pi: S \rightarrow A.$$

- **Target Policy $\pi$:** is the Policy being evaluated and improved, i.e, is the Policy that the agent learns.


- **Behaviour Policy $b$:** is the agent's behavioral function used for action selection during off-policy learning.


- **Deterministic Policy ($\pi(s_t)$)**: the policy always gives the same action for the same input state. The action is computed according to: 

$$a_t = \pi(s_t) \equiv \mu(s_t) \in {\rm I\!R}^{d_a}.$$


- **Stochastic Policy ($\pi(a_t |s_t)$):** is a conditional probability distribution over actions, from which actions are sample from according to: 

$$a_t \sim \pi(a_t |s_t) = \mathbb{P}[A_t = a_t | S_t = s_t] \in {\rm I\!R}^{d_a}.$$ 

The two most common types of stochastic policies are the `categorical policies` and the `diagonal Gaussian policies`. 

- **Parameterized Deterministic Policy ($\pi_{\theta}(s_t)$):** the policy is represented by a neural network (NN) in either discrete or continuous action space. The action is computed according to: $$a_t = \pi_{\theta}(s_t) \doteq \mu_{\theta}(s_t) \equiv \pi (s_t, \theta) \in {\rm I\!R}^{d_a}.$$
    - **Discrete action space:** in practice, it is more common to use stochastic policies for discrete action spaces and deterministic policies for continuous action spaces.
    - **Continuous action space:** in Actor-Critic NNs, the final layer of the Actor network would have `act_dim` $=n$ `neurons` to bound the action space corresponding to a $n$-dimensional vector. For example, each component of the vector could be a torque applied to a joint (actuator) of a robotic arm with values in the range $(-1,1)$. The final layer usually has a `linear or Tanh activation function`. The final layer of the Critic network would have only one neuron (and linear activation function) corresponding to the State-Action value.
        - `Algorithms:` DDPG, TD3.

- **Parameterized Stochastic Policy ($\pi_{\theta}(\cdot |s_t)$):** the policy is represented by a neural network (NN) in either discrete or continuous action space. The final layer of the NN is a conditional distribution. The action sample is computed according to: $$a_t \sim \pi_{\theta}(\cdot |s_t) \doteq \pi(a_t |s_t, \theta) \in {\rm I\!R}^{d_a}.$$
  - **Categorical Policy (discrete action spaces):** uses a neural network (NN) that maps input states to a vector of logits for each action followed by a `softmax activation function` to convert logits into probabilities (categorical distribution). Just like in a classifier, the `number of neurons in the last layer of the NN is equal to the number of actions`.
    - `Algorithms:` TRPO, PPO, A2C.

  - **Diagonal Gaussian Policy (continuous action spaces):** uses a neural network that maps input states to a mean value $\mu_{\theta}(s)$ and a vector of log standard deviations, log $\sigma_{\theta}(s) \in [-\infty, \infty]$, representing the diagonal covariance matrix $\Sigma$ of a diagonal Gaussian distribution.
    - `Algorithms:` TRPO, PPO, A2C, SAC.  
    - **Example 1:** Consider, for example, the problem of a self-driving car where actions such as Steering Angle and Throttle are modeled as continuous values. A Parametrized Stochastic Policy network would contain two pair of neurons in the last layer. Each pair contains one neuron for the mean and another for the variance. 
      - `The activation function for the mean is often Tanh`, to constrain the actions such as steering or throttle within a specific range (e.g., -1 to 1).
      - The variance is constrained to be positive, typically using an `exponential or softplus activation function`.
      - The mean and variance (or equivalently the covariance matrix for multiple actions) define a `multivariate normal distribution`. The variance is related to the spread of the distribution, while the mean determines its central location. The network samples actions from this distribution during training.
    - **Example 2:** the policy for a single robotic arm can have two neurons in the output layer of the NN, one neuron for the mean (representing the desired action) and another neuron for the standard deviation (representing the degree of stochasticity or uncertainty in the action).
    - The action sample is computed according to: $$a = \mu_{\theta}(s) + \sigma_{\theta}(s) \odot z .$$
        - $z \sim \mathcal{N}(0, I)$ denotes a vector of noise from a spherical Gaussian distribution.
        - $\odot$ denotes the element-wise product between two vectors.

## 4.15 Value Functions

The value function gives the expected cumulative future reward a.k.a expected return an agent can get. The expectation is used to yield a normalized estimate of the reward. There are four types of value functions, as outlined below.

**Note:** value functions with `time-dependence` denotes the use of `finite-horizon undiscounted return`, while value functions `without time-dependence` denotes the use of `infinite-horizon discounted return`.

- **Action-Value Function a.k.a state-action value function $Q^{\pi}(s_t, a_t)$:** gives the expected cumulative future reward a.k.a expected return an agent will get if starting from state $s_t=s_0$, taking arbitrary action $a_t=a_0$ (not necessarily from the Policy), and then forever acting under Policy $\pi$. In other words, it evaluates how good is action $a_t \in A$ in state $s_t \in S$.

$$Q^{\pi}(s_t, a_t): S \rightarrow {\rm I\!R}.$$
$$Q^{\pi}(s_t, a_t) = \mathbb{E}_{\tau \sim \pi}[\mathcal{R}(\tau) | s_0 = s_t, a_0 = a_t] = \frac{1}{N}\sum_{n=1}^N \mathcal{R}_t^n.$$

When the Policy function is implicit, i.e, when the agent does not have direct access to the Policy, actions are computed from the $Q$-value according to:

$$a(s_t)=\underset{a}{\operatorname{arg\,max}} \ Q(s_t, a_t) \text{ (Discrete Action space)}.$$

$$a(s_t) \approx \text{arg} \ Q(s_t, \mu(s_t)) \text{ (Continuous Action space)}.$$

In this case, the Policy is deterministic.

- **State-Value Function $V^{\pi}(s_t)$:** gives the expected cumulative future reward a.k.a expected return from state $s_t = s$ onwards while acting according to Policy $\pi$.

$$V^{\pi}(s_t): S \rightarrow {\rm I\!R}.$$
$$V^{\pi}(s_t) = \mathbb{E}_{\tau \sim \pi}[\mathcal{R}(\tau) | s_t = s].$$
$$V^{\pi}(s_t) = \mathbb{E}_{a_t \sim \pi(s_t)}[Q^{\pi}(s_t, a_t)].$$


- **Optimal Action-Value Function $Q^*(s_t, a_t)$:** gives the expected cumulative future reward a.k.a expected return an agent will get if starting from state $s_t=s$, taking arbitrary action $a_t = a$, and then forever acting under the optimal Policy.

    $$Q^*(s_t, a_t) = \underset{\pi}{\operatorname{max}} \ \mathbb{E}_{\tau \sim \pi} [\mathcal{R}(\tau) |s_t = s, a_t = a].$$
    $$a^*(s_t) = \underset{a}{\operatorname{arg\,max}} \ Q^*(s_t, a_t).$$
    
    
- **Optimal State-Value Function $V^*(s_t)$:** gives the expected cumulative future reward a.k.a expected return from state $s_t = s$ onwards while acting according to the optimal Policy.

$$V^*(s_t)=\underset{\pi}{\operatorname{max}} \ \mathbb{E}_{\tau \sim \pi}[\mathcal{R}(\tau)|s_t = s].$$
$$V^*(s_t)=\underset{a}{\operatorname{max}} \ Q^*(s_t, a_t).$$

## 4.16 Bellman Equations

Bellman equations correspond to a set of four self-consistency equations.

- **The state-value function can be decomposed into:**

$$V^\pi(s) = \underset{\underset{a \sim \pi}{s' \sim \mathcal{P}}}{\mathbb{E}} \ [r(s, a) + \gamma V^{\pi}(s') | S = s].$$

- **The Q-value function can be decomposed into:**

$$Q^{\pi}(s, a) = \underset{s' \sim \mathcal{P}}{\operatorname{\mathbb{E}}} \ \Bigg[r(s,a) + \gamma \mathbb{E}_{a'\sim\pi}\Big[Q^{\pi}(s', a')\Big] | S = s, A=a\Bigg].$$


- **Bellman equation for the optimal state-value function:**

$$V^*(s) = \underset{a}{\operatorname{max}} \ \underset{s' \sim \mathcal{P}}{\operatorname{\mathbb{E}}} \ [r(s, a) + \gamma V^*(s') | S = s].$$


- **Bellman equation for the optimal Q-value function:**

$$Q^*(s, a) =\underset{s' \sim \mathcal{P}}{\operatorname{\mathbb{E}}} \ \Bigg[r(s,a) + \gamma \underset{a'}{\operatorname{max}} \ \Big[Q^*(s', a')\Big] | S = s, A=a\Bigg].$$

- **Bellman backup:** is the term in the right-hand side (RHS) of the Bellman equation.

**Proof:**

\begin{eqnarray}
V(s) &=& \mathbb{E} [\mathcal{R}_t| S_t = s] \\
&=& \mathbb{E} [r_t + \gamma r_{t+1} + \gamma^2 r_{t+2} + \gamma^3 r_{t+3} + ... | S_t = s] \\
&=& \mathbb{E} [r_t + \gamma  (r_{t+1} + \gamma r_{t+2} + \gamma^2 r_{t+3} + ...) | S_t = s] \\
&=& \mathbb{E} [r_t + \gamma  \mathcal{R}_{t+1} | S_t = s] \\
&=& \mathbb{E} [r_t| S_t = s] + \mathbb{E}[\gamma \mathcal{R}_{t+1} | S_t = s] \  \color{green}{\text{(using property 1).}} \\
&=& \mathbb{E} [r_t| S_t = s] + \gamma \mathbb{E}[\mathcal{R}_{t+1} | S_t = s] \ \color{green}{\text{(using property 2).}}\\
&=& \mathbb{E} [r_t| S_t = s] + \gamma V(S_{t+1}) \ \color{green}{\text{(using the definition of state-value function).}}\\
&=& \mathbb{E} [r_t | S_t = s] + \mathbb{E}[\gamma V(S_{t+1})| S_t = s] \ \color{green}{\text{(using property 3).}}\\
&=& \mathbb{E} [r_t + \gamma V(S_{t+1}) | S_t = s] \ \color{green}{\text{(using property 1).}} 
\end{eqnarray}

Where it was used $\mathbb{E}[G_{t+1}|S_t=s] = V(S_{t+1})$ and the following properties:

1. The expected value of a sum is the sum of the expectations: 

$$\mathbb{E}[X+Y] = \mathbb{E}[X]+ \mathbb{E}[Y].$$ 


2. The expected value with a constant is the constant multiplied by the expected value: 

$$\mathbb{E}(\gamma X) = \gamma \mathbb{E}(X).$$


3. [Law of iterated expectations a.k.a law of total expectation](https://en.wikipedia.org/wiki/Law_of_total_expectation): the expected value of an expected value is 

$$\mathbb{E}[\mathbb{E}[X]]=\mathbb{E}[X],$$ 

with consequences such as $$\mathbb{E}(2X\mathbb{E}(X)) = 2E(X\mathbb{E}(X))=2\mathbb{E}(X)\mathbb{E}(X),$$

which can be used to prove the identity

$$\text{Var}(X) = \mathbb{E}((X-\mathbb{E}(X))^2) = \mathbb{E}(X^2) - (\mathbb{E}(X))^2.$$

Using Bellman equation, one can write:

$$V(s)= r +\gamma \sum_{s' \in S} \mathcal{P}{ss'}V(S'),$$

and also

$$V= r+\gamma \mathcal{P}V.$$

This is a linear equation whose solution is also the solution for the MDP:

\begin{eqnarray}
V &=& r+\gamma \mathcal{P}V \\ 
&\implies& r = V-\gamma \mathcal{P}V \\
&\implies& r = (\mathcal{I}-\gamma \mathcal{P})V \\
&\implies& (\mathcal{I}-\gamma \mathcal{P})^{-1} r = (\mathcal{I}-\gamma \mathcal{P})^{-1}(\mathcal{I}-\gamma \mathcal{P})V = V
\end{eqnarray}

The computational complexity for calculating the inverse matrix is $\mathcal{O}(n^3)$, meaning that direct solution is only possible for small MRP.

The Bellman Optimality Equation is non-linear and, therefore, cannot be solved by linear algebra methods such the previous matrix inversion. And, in general, it has no [closed form solution](https://en.wikipedia.org/wiki/Closed-form_expression). To solve this recursive equation, one needs iterative methods (dynamic programming) such as:
- Value iteration (on-Policy learning).
- Policy iteration (on-Policy learning).
- Q-learning (almost always off-policy learning).
- Sarsa (on-Policy learning).

## 4.17 Advantage Function

- **Advantage Function:** 

$$A^{\pi}(s_t, a_t) = Q^{\pi}(s_t, a_t) - V^{\pi}(s_t).$$

The Advantage function is the difference between the expected return (Action-Value function) and the baseline estimate. It computes an estimate of the "relative value of the selected action". The baseline is the State Value function that gives a noisy estimate of the return.

## 4.18 Exploitation vs Exploration: the RL tradeoff

- **Exploitation:** chooses the action that gives the maximum reward for the current time step, i.e, uses known information (collected experiences) to maximize reward choosing the best actions.
- **Exploration:** sacrifices reward to find more information about the environment.

## 4.19 $\epsilon$-greedy policy

The $\epsilon$-greedy policy is used in discrete action spaces.

**Algorithm (adapted from Hado van Hasselt, UCL):**

- Generate a random number $\epsilon \in [0,1]$.

- Sample a greedy action (exploitation) with probability $1-\epsilon$:

$$a(s_t)=\underset{a \in A}{\operatorname{arg\,max}} \ Q_{\tau}(s_t, a_t) \text{ (Discrete Action space)}.$$

$$a(s_t) \approx \text{arg} \ Q(s_t, \mu(s_t)) \text{ (Continuous Action space)}.$$

- Else, sample a random action (exploration) with probability $\epsilon$.

**Note:** to improve exploration, the parameter $\epsilon$ can be decreased over time (epsilon decay). A constant $\epsilon$ may lead to suboptimal action (linear expected total regret).


Summary:

- Greedy policy is deterministic.

- $\epsilon$-greedy policy is stochastic.
 
\begin{equation}
\pi_t(a_t) = \begin{cases}
  (1-\epsilon)+\epsilon/|A|, & \text{if }  Q_{\tau}(s_t, a_t) = \underset{b \in B}{\operatorname{max}}Q_{\tau}(b_t).\\
  \epsilon/|A|, & \text{otherwise. }\\
 \end{cases}
\end{equation}

Even though DQN uses $\epsilon$-greedy policy (stochastic) during exploration, the policy produced by DQN after training (used in production/deployment) is deterministic. 

## 4.20 Importance sampling

A concept used in off-Policy learning to compute the expectation value of the return $\mathcal{R}$ under the target Policy $\pi(a|s)$ using reward signals sampled from the behavior Policy $b(a|s)$:

$$\mathbb{E}_{\pi} [\mathcal{R}(\tau)] = \sum_{r\in\mathcal{R}(\tau)} r \pi(a|s) = \sum_{r\in\mathcal{R}(\tau)} r \pi(a|s) \frac{b(a|s)}{b(a|s)} =\sum_{r\in\mathcal{R}(\tau)} r \rho b(a|s)= \mathbb{E}_b [\mathcal{R}(\tau) \rho] \approx \frac{1}{n}\sum_{i=0}^{n-1}r_i \rho_i.$$

Where

$$ \rho = \frac{\rho_0(s_0) \prod_{t=0}^{T-1}\mathbb{P}(r_{t+1}, s_{t+1}|s_t,a_t) \cdot \pi(a_t | s_t)}{\rho_0(s_0) \prod_{t=0}^{T-1}\mathbb{P}(r_{t+1}, s_{t+1}|s_t,a_t) \cdot b(a_t | s_t)} = \prod_{t=0}^{T-1} \frac{\pi(a_t | s_t)}{b(a_t | s_t)},$$


and $n$ denotes the number of time steps in a trajectory.

## 4.21 Bootstrapping

**In statistics**: bootstrapping refers to a statistical technique that consists of `sampling with replacement` while pretending that the sample dataset is the population dataset.

- Problem: suppose we want to estimate the average height of people in a large population. It is not feasible to measure the height of each person individually.
    - First solution: use Monte Carlo by sampling 1000 people randomly and then taking the average of the sample dataset. The mean average will be closer to the population average given the law of large numbers.
    - Second solution: use Bootstrapping. Pretend that the sample dataset is the population dataset, then apply the Monte Carlo method, i.e., sample 1000 people randomly, and then take the average of the sample dataset. Now, repeat (say 100 times) the same procedure with replacement. In the end, there will be 100 estimates of the height. Finally, take the standard deviation from those estimates.
    


**In reinforcement learning:** the agent updates its current value by using estimates/predictions of future values.

Temporal Difference (TD) learning is a bootstrapping method. Example: 

$$Q(s,a) \leftarrow Q(s,a) + \alpha(R_{t+1}+\gamma Q(s',a')-Q(s,a)),$$

where $R_{t+1}+\gamma Q(s',a')$ is known as the TD target Q-value.

## 4.22 Deadly triad issue

The deadly triad is a drawback caused by instabilities and/or divergence in the value function when using bootstrapping, off-policy learning, and function approximations such as neural networks in the same algorithm.

Circumventions: target networks, deep double q-learning, and prioritized replay.

# 5. Taxonomy (extended)<a name="tax" />

Main reference for this section: [John Schulman's PhD Thesis](https://www2.eecs.berkeley.edu/Pubs/TechRpts/2016/EECS-2016-217.html), [David Silver's RL course](https://www.davidsilver.uk/teaching/), and [Sergey Levine's DRL course](http://rail.eecs.berkeley.edu/deeprlcourse/).

This section is dedicated to understand the below taxonomy from [Open AI](https://spinningup.openai.com/en/latest/spinningup/rl_intro2.html#a-taxonomy-of-rl-algorithms):

![Taxonomy](../assets/taxonomy.svg)
© Image Credits: [Open AI](https://spinningup.openai.com/en/latest/spinningup/rl_intro2.html#a-taxonomy-of-rl-algorithms).

- **Model-based RL:** there is a model of the environment for planning (lookahead), i.e, a function to predict future situations (state transitions and rewards) before they actually happen. Algorithms of this kind are more sample efficient. The agent can learn either the `Policy` and/or the `Value function`.


- **Model-free RL:** methods are based explicitly in trial-and-error via interaction with the unknown environment. There is no model of the environment. Algorithms of this kind are less sample efficient, but easier to implement and tune. The agent can learn either the `Policy` and/or the `Value function`.

- **Dynamic Programming (DP) Methods:** DP is an optimization method developed by Richard Bellman (1950) that consists in splitting a complex problem into its subproblemms with optimal substructure. In RL, it exploits the structure of the problem indirectly to learn a `Value Function` using self-consistency properties of the same. Algorithms of this class are more sample-efficient and compatible with exploration and off-policy learning.
    - **Policy Iteration:** is a `model-based` (uses a transition probability) `on-policy` (no replay buffer) and non-parametric `tabular method` in RL (data is stored in a `look-up table`) that solves (exactly) the true `Optimal Policy function` $\pi^*$ (`Policy is deterministic`) for a Markov Decision Process (MDP). The algorithm has two interacting steps in a feedback loop until Policy converges: Policy evaluation followed by Policy improvement. The algorithm starts with a random Policy and random value function until convergence via a series of updates.
        - `Policy evaluation:` updates the State-Value function $V^{\pi}(s)$ given the current Policy $\pi$. One advantage over Value iteration (nonlinear due to max operator) is that this step can be computed with a linear program without the use of dynamic programming.
        - `Policy improvement:` updates the current Policy $\pi$ given the updated State-Value function $V^{\pi}(s)$. 
    - **Value Iteration:** is a `model-based` (uses a transition probability) `on-policy` (no replay buffer) and non-parametric `tabular method` in RL (data is stored in a `look-up table`) that iteratively computes (exactly) the true `optimal State-Value function` $V^*(s)$ for an MDP and extracts the optimal Policy (`Policy is deterministic`) only once using the Bellman optimality equation. Consists of two main steps: `finding optimal State-Value function` and `one Policy extraction`. The algorithm can also be modified to solve for the Action-Value function. The algorithm starts with a random value function and through an iterative process converges to the true optimal value function within a margin (threshold) of error. Finally, the optimal Policy is extracted using Bellman optimality equation. The algorithm requires dynamic programming since the optimality Bellman operator is nonlinear due to the max operator.
        - **Q-learning:** approximate version of value iteration. Is a `model-free` (no state transition probability distribution) `off-policy` (uses a replay buffer), `temporal difference` (TD uses bootstrapping) and `tabular method` in RL (data is stored in a look-up table) that computes the `optimal Q-function` $Q^*(s_t, a_t)$ for each state-action pair. The Q-function is represented by a look-up table of Q-values for each state-action pair. It learns an implicit greedy Policy (deterministic) using a behaviour Policy (usually $\epsilon$-greedy, i.e, stochastic). The objective function is based on the optimal Bellman equation.
            - Examples of algorithms: C51.
        - **SARSA:** `on-policy` value-based Temporal Difference (TD uses bootstrapping) control algorithm that combines Monte Carlo and dynamic programming methods.
    - **Modified Policy Iteration:** the Policy evaluation step is repeated for $k$ times update.

- **Policy Optimization Methods:** optimize the Policy parameters `either directly`, computing the `gradient ascent` of the performance objective $J(\pi_{\theta})$ (a.k.a expected return under parameterized policy), `or indirectly`, by maximizing `local approximations` of $J(\pi_{\theta})$. Algorithms of this class are compatible with nonlinear function approximators, such as multilayer perceptron neural networks (MLP) and recurrent neural networks (LSTM). MLP-based implementations, since they lack convolutional neural networks, are more suitable for fully-observed, non-image-based RL environments. Algorithms of this class are also more stable (less variance) than Q-learning, however, they tend to be less sample efficient, as they are `almost always based in on-policy` learning. Examples of algorithms: DFO, hill climbing, finite difference methods, likelihood ratio policy gradients.
    - **Derivative-free Optimization (DFO) a.k.a Evolutionary algorithms:** treats the entire process of reward generation as a black box and optimize over the policy parameter vector $\theta_i$ (that follows some distribution such as a Gaussian) to maximize the return. The inputs ($\theta_i$) to the black box can be chosen, the outputs (rewards) can only be observed. Algorithms of this class do not scale well with the number of parameters.
        - Examples of algorithms: 
            - Cross Entropy Method (CEM): learns a parameterized Policy with a neural network without backpropagation.
    - **Policy Gradient:** measures the `gradient of the policy performance objective` $J(\pi_{\theta})$ with respect to the parameters $\theta$ of a parameterized Policy $\pi_{\theta}$ that is approximated/represented by a `neural network` as the expressive nonlinear function approximator. The Policy could be either `deterministic` or `stochastic`. Algorithms of this class scale well with the number of parameters.
        - Examples of algorithms: 
            - Finite Difference Methods.            
            - VPG: learns On-Policy value function and On-Policy stochastic Policy.
            - And the following actor-critic methods: A2C, A3C, DPG, DDPG, D4PG, MADDPG, TRPO, PPG, TD3, and SVPG.

- **Deep RL:** uses a `deep neural network as an expressive nonlinear function approximator` to learn either a `parameterized value function`, a `parameterized Policy`, `or the model`. The optimization of the value function / policy / model can be done `end-to-end with stochastic gradient descent`.
    - **Deep Q-Network (DQN):**  developed by DeepMind in 2015, is the enhanced version of Q-learning. Is a model-free off-policy (uses a replay buffer) approach for large-scale RL systems. It learns a parameterized $Q_{\theta}(s_t, a_t)$ function represented by a `single neural network` (function approximator) as opposed to a look-up table to estimate the `parameterized optimal Action-Value function` $Q^*_{\theta}(s_t, a_t)$ for a state-action pair. The Q-values of the parameterized Q-function are estimated by the neural network. To ensure stability, it uses `experience replay` (transitions are added to a replay buffer at each time step) and a snapshot of previous transitions known as the `frozen target network`. It also uses $\epsilon$-`greedy approach` to ensure `exploitation` and for action selection. The NN's forward pass is then computed with a `mini-batch` of transitions sampled from the replay buffer to find the gradient of the loss function. The loss function is encoded by the `Bellman residual`. The original DQN is for `discrete action space`. The number of neurons in the input layer equals the observation space, while the number of neurons in the output layer equals the number of actions. The `output layer uses a linear activation function`. After training, the action given the Policy (`Policy is deterministic`) is obtained according to: 
    $$a^*(s_t) \in {\rm I\!R}^{d_a} = \underset{a}{\operatorname{arg\,max}} \ Q_{\theta}^*(s_t, a_t).$$ `Remember: time-dependence indicates finite-horizon undiscounted return`. DQN is preferable over tabular methods such as Q-learning and Value/Policy iteration which are time-consuming given the large state space required to iterate over and unfeasible (memory bottlenecks) to store values in a look-up table as the state-action space grows. Learning look-up tables is replaced by learning weights in the neural network. ["Training a deep RL agent on 50+ Atari 2600 games in ALE for 200M frames (the standard protocol) requires 1,000+ GPU days."](https://ai.googleblog.com/2022/11/beyond-tabula-rasa-reincarnating.html?m=1)
    - **Actor-Critic Methods:** a combination of `Policy Gradient` methods and `Policy/Value Iteration` methods that uses `two neural networks`, the Actor and the Critic, as expressive nonlinear function approximators.
        - Examples of algorithms:
            - A2C: is an On-Policy temporal difference (TD) learning method that uses a stochastic Policy (Actor network). Continuous or discrete action space.
            - A3C: uses On-Policy learning. Continuous or discrete action space.
            - DPG. there is On-Policy and Off-Policy versions. Continuous action space.
            - DDPG: is a model-free (no transition probability) off-policy `actor-critic` algorithm that combines elements of policy gradient methods with deep Q-learning. DDPG is an extension of DQN for continuous action space. It uses `temporal difference learning` (bootstrapping) and `experience replay buffer` (off-policy) to learn the Q-value (represented by the Critic network). Unlike DQN, DDPG does not use $\epsilon$-greedy policy (exploitation) for action selection. Rather, In DDPG, the behavior policy for action selection is derived from the actions generated by the Actor network (which is a deterministic target policy) with the addition of noise to encourage `exploration` in the environment.
            - D4PG: uses Off-Policy learning. Continuous action space.
            - MADDPG: uses Off-Policy learning. Continuous action space.
            - TRPO: uses On-Policy learning and stochastic Policy (Actor network). Continuous or discrete action space.
            - PPG: learns an Off-Policy value function (Critic network) and an On-Policy Policy function (Actor network).
            - TD3: uses Off-Policy learning and deterministic Policy (Actor network). Continuous action space. Addresses DDPG's lack of stability.
            - PPO: uses On-Policy learning and stochastic Policy. Continuous or discrete action space. Optimization indirectly maximize the performance object, instead uses a surrogate objective.
            - SAC: uses Off-Policy learning and stochastic Policy. Continuous action space. Maximizes expected reward and entropy. Addresses DDPG's lack of stability.
            - ACER: uses Off-Policy learning. Continuous or discrete action space.
            - ACTKR: uses On-Policy learning. Continuous or discrete action space.

# 6. Open Challenges in RL<a name="open" />

- **Sparse rewards:** is when the agent only gets a reward after the end of the episode, since it does not know which of the actions in the trajectory were the cause of negative scores. Arises from the fact that we don't know the target labels that the network should produce from the input data, so the agent should learn from sparse feedback.
    - **Reward shaping:** one solution to the `sparse rewards` problem is to craft a new reward function at each time step. However, this approach is not scalable as each new environment will need a tailored reward.

- **Alignment problem:** the `reward shaping` suffers from the `alignment problem`, i.e, the policy will overfit to the specific reward function without generalizing or behaving as expected.
    
    
    
- **Reinforcement Learning with sparse rewards:**
  - [Auxiliary Reward signals](https://arxiv.org/abs/1611.05397): augment the sparse reward signal coming from the environment by additional feedback signal.
  - [epsilon greedy exploration](#): choose a random action with probability $p$ or pick the top action from the policy network with probability $1-p$. Does not work when the environment is too complex to explore.
  - [Curiosity-driven exploration by self-supervised prediction](https://arxiv.org/abs/1705.05363): incentivize exploration in the environment with a forward model, i.e, encode the input state (observation) into a latent representation and then use the foward model to predict this same representation for the next frame of the env. It tries to learn the dynamics of the env. This should be used as an additional feedback signal on top of the sparse reward.
  - [Hindsight Experience Replay (HER)](https://arxiv.org/abs/1707.01495): the idea is to make the agent learn even from unsuccessful episodes by creating a dense reward setting with modified virtual goals that will turn a failure into a sucess.

# 7. Policy Optimization Algorithms<a name="algo" />

Main reference for this section: [Sutton & Barto 2018 Chap. 13 c.f. [4]](#), [Open AI](https://spinningup.openai.com/en/latest/spinningup/rl_intro3.html), [John Schulman's PhD Thesis](https://www2.eecs.berkeley.edu/Pubs/TechRpts/2016/EECS-2016-217.html), and [Sergey Levine's DRL course](http://rail.eecs.berkeley.edu/deeprlcourse/).

## 7.1 Derivative-free Optimization Algorithms

Gradient-based reinforcement learning might be overkill to some tasks, therefore, it is worth to first investigate derivative-free optimization algorithms such as the Cross Entropy Method (CEM).

### 7.1.1 Cross Entropy Method (CEM)

The Cross Entropy Method (CEM) is one example of evolutionary algorithm that works with Gaussian distributions through successful updates of the mean and variance. CEM shows better performance in the game Tetris (used as task benchmark) than Policy Gradient Methods. This may be due to the discount factor that limits the scalability of the action space since further rewards are less valuable. 

---
**Algorithm (Pseudocode): Cross Entropy Method Algorithm (adapted from John Schulman)**

---

- Input: initialize two $d$-dimensional vectors: generate a Gaussian distribution with mean $\mu \in \mathbb{R}^d$, and standard deviation $\sigma \in \mathbb{R}^d$.

- for iteration = 1, 2, ... , n do:

    - Collect $n$ samples $\theta_i$ (the weights a.k.a parameters of the Policy) from the Gaussian distribution, i.e, $\theta_i ∼ N(μ, diag(σ))$.
    
    - Run one episode with each $\theta_i$ and collect the return $R_i \sim \theta_i$.
    
    - Select the top $p$% of samples (e.g. $p = 20$) and store then into an "elite set".
    
    - Fit a Gaussian distribution, with diagonal covariance, to the elite set, obtaining the new mean $\mu$, and standard deviation $\sigma$.
    
- end for.

- Return final mean $\mu$.

---

## 7.2 [Policy Gradient Algorithms](https://spinningup.openai.com/en/latest/spinningup/rl_intro3.html)

The goal of reinforcement learning is to maximize the expected cumulative reward (a.k.a expected return) under policy $\pi_{\theta}$:

$$\text{max } J(\pi_{\theta}) = \text{max } \mathbb{E}_{\tau \sim \pi_{\theta}}[\mathcal{R}(\tau) | \pi_{\theta_t}].$$

In Policy Gradient Algorithms, this can be achieved by directly updating/optimizing the parameters $\theta_t$ of the parameterized Policy $\pi_{\theta}$ computing the `gradient ascent` of the performance objective $J(\pi_{\theta})$ with respect to the Policy parameters $\theta_t$:


\begin{eqnarray}
\theta_{t+1} &=& \theta_t + \alpha \nabla_{\theta} J(\pi_{\theta})|_{\theta_t}\\
&=& \theta_t + \alpha \nabla_{\theta} \mathbb{E}_{\tau \sim \pi_{\theta}}[\mathcal{R}(\tau) | \pi_{\theta_t}].
\end{eqnarray}


Where:

-  $\pi_{\theta}$ is the parameterized Policy function represented by a neural network as an expressive nonlinear function approximation.

- $\alpha$ is the learning rate.
  
- $\nabla_{\theta} J(\pi_{\theta})$ denotes the gradient of `policy performance` a.k.a `policy gradient`.

- $\mathcal{R}(\tau) \doteq \sum_{t=0}^T r_t \in {\rm I\!R} \text{ (Finite-horizon undiscounted return)}$ is the sum of rewards over a fixed window of time steps.

- $\mathcal{R}(\tau) \doteq \sum_{t=0}^{\infty} \gamma^t r_{t} \in {\rm I\!R} \text{ (Infinite-horizon discounted return)}$ is the sum of all rewards ever obtained.

- $\gamma \in [0,1]$ is the discount factor.

Considering the infinite-horizon discounted return function, the performance objective $J(\pi_{\theta})$ is defined as the expected cumulative reward under a parameterized policy $\pi_{\theta}$, which is mathematically represented as:

$$J(\pi_{\theta}) =  \mathbb{E}_{\tau \sim \pi_{\theta}}[\mathcal{R}(\tau)] = \int_{\tau} \underbrace{\mathbb{P}(\tau | \pi_{\theta})}_{\text{Trajectory prob.}}  \underbrace{\mathcal{R}(\tau)}_{\text{Return}},$$

with the the probability of trajectory $\tau$ under the policy $\pi_{\theta}$ given by

$$\mathbb{P}(\tau | \pi_{\theta}) = \underbrace{\rho_0(s_0)}_{\text{start-state dist.}} \prod_{t=0}^{T-1} \underbrace{\underbrace{\mathbb{P}(s_{t+1}|s_t,a_t)}_{\text{State transition prob.}}}_{\text{Env. model.}} \cdot \underbrace{\underbrace{\pi(a_t | s_t)}_{\text{Action prob.}}}_{\text{Control function.}}.$$


Legend:

- $\tau = (s_0, a_0, s_1, a_1, \dots)$ represents a trajectory (a sequence of states and actions).

-  $\pi_{\theta}$ is the parameterized Policy function represented by a neural network as an expressive nonlinear function approximation.

- $\mathbb{P(\tau | \pi_{\theta})}$: is the probability of getting a trajectory $\tau$ with $T$ time steps acting according to policy $\pi_{\theta}$.

- $\rho_0$ denotes the initial (start)-state probability distribution. The initial state $s_0$ is sampled from the probability distribution: $s_0 \sim \rho_0(\cdot).$

- $\mathcal{R}(\tau) = \sum_{t=0}^{T} \gamma^t r_t$ is the cumulative (discounted) reward for the trajectory over an episode.

- $\gamma \in [0, 1]$ is the discount factor.

To compute the policy gradient numerically, one must first derive an analytical expression for the policy gradient in terms of the expected value and then sample trajectories through agent-environment interaction steps. The policy gradient reads:

\begin{eqnarray}
\nabla_{\theta} J (\pi_{\theta}) &=& \nabla_{\theta} \mathbb{E}_{\tau \sim \pi_{\theta}}[\mathcal{R}(\tau)] \quad (Eq. 1.1) \\
&=& \nabla_{\theta} \int_{\tau} \mathbb{P}(\tau | \pi_{\theta}) [\mathcal{R}(\tau)] \quad (Eq. 1.2) \\
&=& \int_{\tau} \nabla_{\theta} \mathbb{P}(\tau | \pi_{\theta})  [\mathcal{R}(\tau)] \quad (Eq. 1.3) \\
&=& \int_{\tau}  \mathbb{P}(\tau | \pi_{\theta}) \Bigg[ \nabla_{\theta} log(\mathbb{P}(\tau | \pi_{\theta}))  \mathcal{R}(\tau) \Bigg] \quad (Eq. 1.4) \\
&=& \mathbb{E}_{\tau \sim \pi_{\theta}} \Bigg[ \nabla_{\theta} log(\mathbb{P}(\tau | \pi_{\theta}))  \mathcal{R}(\tau) \Bigg] \quad (Eq. 1.5) \\
&=& \mathbb{E}_{\tau \sim \pi_{\theta}} \left[ \sum_{t=0}^T\nabla_{\theta}log(\pi_{\theta}(a_t| s_t))\mathcal{R}(\tau) \right] \quad (Eq. 1.6)\\
&\approx& \frac{1}{|\mathcal{D}_k|}\sum_{\tau \in \mathcal{D}_k}\sum_{t=0}^T\nabla_{\theta}log(\pi_{\theta}(a_t| s_t))|_{\theta_k}\mathcal{R}(\tau) \quad (Eq. 1.7).
\end{eqnarray}

$\quad$

- Eq. 1.2 uses the definition of the expected return, noting that $\mathbb{E} [\cdot] = \int \mathbb{P}(\cdot) [\cdot]$.

<div style="background-color: yellow; padding: 10px; border-radius: 5px;">

**Expected value (a.k.a population mean or weighted average)  of a distribution:**

- Expected value for **discrete variables**:
    $$\mu \doteq  \mathbb{E}[X]  \doteq \langle X\rangle = \sum_{j=1}^{\dim \Omega = d} x_jP(x_j).$$
    The expected value is not the most likely value of $X$ and may not even be a possible value of $X$, but it is bound by
    $$X_{\min} \leq\langle X\rangle\leq X_{\max}.$$

- Expected value for **continuous variables**:
    $$\langle X \rangle = \int_{-\infty}^{+\infty} x \rho(x) dx.$$
  
</div>
  
- Eq. 1.3 uses the [Leibniz integral rule](https://en.wikipedia.org/wiki/Leibniz_integral_rule) to bring the gradient symbol under the integral sign. This is possible because the integration domain (over $\tau$) does not depend on $\theta$. 
  
- Eq. 1.4 uses the log-derivative trick:

  \begin{eqnarray}\
  \frac{d}{dx}ln(f(x)) &=& \frac{1}{f(x)}\frac{d}{dx}f(x). \\
                         &\rightarrow&  \nabla_{\theta} log(\mathbb{P}(\tau | \pi_{\theta})) = \frac{1}{\mathbb{P}(\tau | \pi_{\theta})}  \nabla_{\theta} \mathbb{P}(\tau | \pi_{\theta})\\
                       &\rightarrow&  \nabla_{\theta} \mathbb{P}(\tau | \pi_{\theta}) =  \mathbb{P}(\tau | \pi_{\theta}) \nabla_{\theta} log(\mathbb{P}(\tau | \pi_{\theta}))
  \end{eqnarray}

- Eq. 1.5 is the expectation form of Eq. 1.4 using $\mathbb{E} [\cdot] = \int \mathbb{P}(\cdot) [\cdot]$.

- Eq. 1.6 computes the expected value of the **gradient of the log-probability of a trajectory**, weighted by the return $\mathcal{R}$, over all trajectories sampled i.i.d from the policy.

<div style="background-color: yellow; padding: 10px; border-radius: 5px;">

**Derivation:**

  \begin{eqnarray}
   \nabla_{\theta} log(\mathbb{P}(\tau | \pi_{\theta})) &=& \nabla_{\theta} log \Bigg( \rho_0(s_0) \prod_{t=0}^{T} \mathbb{P}(s_{t+1}|s_t,a_t) \cdot \pi(a_t | s_t) \Bigg) \\
   &=& \nabla_{\theta} \Bigg[log (\rho_0(s_0)) + log \left( \prod_{t=0}^{T} \mathbb{P}(s_{t+1}|s_t,a_t) \cdot \pi(a_t | s_t) \right) \Bigg]\\
   &=& \nabla_{\theta} \Bigg[log (\rho_0(s_0)) + \sum_{t=0}^{T} log \Big( \mathbb{P}(s_{t+1}|s_t,a_t) \cdot \pi(a_t | s_t) \Big) \Bigg]\\
   &=& \nabla_{\theta} \Bigg[log (\rho_0(s_0)) + \sum_{t=0}^{T} log \Big(\mathbb{P}(s_{t+1}|s_t,a_t)\Big) + log \Big(\pi(a_t | s_t)\Big) \Bigg]\\
   &=& \Bigg[\cancel{\nabla_{\theta} log (\rho_0(s_0))} + \sum_{t=0}^{T} \cancel{\nabla_{\theta} log \Big(\mathbb{P}(s_{t+1}|s_t,a_t)\Big)} + \nabla_{\theta} log \Big(\pi(a_t | s_t)\Big) \Bigg]\\
   &=& \sum_{t=0}^T\nabla_{\theta}log(\pi_{\theta}(a_t| s_t))
  \end{eqnarray}
</div>

- Eq. 1.7 computes the empirical average (approximation) of Eq. 1.6. It is an unbiased estimator (sample mean) of the expectation $\mathbb{E}_{\tau \sim \pi_{\theta_k}} [\cdot]$ since trajectories are sampled i.i.d from the policy. As the number of trajectories $|\mathcal{D}_k| \rightarrow \infty$, Eq. 1.7 converges to Eq. 1.6 due to the Law of Large Numbers.
 
Legend:

- $\mathcal{D}_k$ denotes the set with a number $|\mathcal{D}_k|$ of trajectories sampled i.i.d from the Policy $\pi_k$ in the $k$-th iteration.
- i.i.d means independent and identically distributed.
    - Independent: the outcome of one random variable does not affect the outcomes of the others.
    - Identically Distributed: all random variables in the collection follow the same probability distribution (e.g., same mean and variance). 

## 7.3 Actor-Critic Algorithms

Actor-Critic (AC) algorithms are a type of temporal difference learning method that make use of two neural networks as expressive nonlinear function approximators:

- **Actor network:** learns/estimates the `parameterized Policy function` $\pi_{\theta_t}$ (**target policy**). It is a mapping from state to actions that is approximated/represented by a `neural network` as the expressive nonlinear function approximator. The goal is to update/optimize the Policy parameters $\theta_t$ using `gradient ascent` in order to `maximize the expected cumulative reward (a.k.a expected return)` provided as feedback from the Critic network.
<br>

- **Critic network:** uses policy evaluation (e.g. `Temporal Difference learning`) to estimate either the Action-Value function $Q^{\pi}(s_t, a_t)$, the State-Value function $V^{\pi}(s_t)$, or the advantage function $A^{\pi}(s_t, a_t)$ under Policy $\pi$. It is trained using `gradient descent` to minimize the difference between its estimated values and the actual observed returns.
<br>

**Note1:** temporal difference means computing the difference of estimated values between successive states.

**Note2:** The critic network is not the Behaviour Policy $b$.
   
Examples of AC algorithms are: A2C, A3C, DDPG, D4PG, MADDPG, TRPO, PPO, TD3, SAC, ACER, ACTKR.

- Actor Network:
    - Input: the input to the Actor network is typically the vector representing the `current state` of the environment.
    - Output: the output of the Actor network is a vector of `actions`.

- Critic Network:
    - Input: the input to the Critic network is a combination of the `current state vector` (sometimes reward) and the output `action vector` from the Actor. These inputs are concatenated or combined in some way to form a joint representation.
    - Output: the output of the Critic network is the `estimated action-value` or Q-value associated with the given state-action pair. It represents the expected cumulative reward when taking the given action in the given state.

# 8. Avoiding memory bottlenecks in deep neural networks <a name='memory' />

1. Prune the neural network model to have less hyperparameters.


2. Quantize the neural network model to have less precision by working with int8 data type instead of float32. Floating point operations cost more than integers. Recall that `np.int8` uses 1 byte and can hold values from -128 to 127, and `np.uint8v` uses 1 byte and hold values from 0 to 255, while `np.float32` uses 4 bytes.


3. Read the dataset from an external HD in batches using python Generator objects instead of loading everything into ram memory (such as in a numpy ndarray, in a set, or worse in a list). There is a tradeoff since computations with list comprehension are faster than with generators. And there's an added overhead from moving data from the HD to the ram memory since this will slow down training.


4. Run experiment with cloud computing: [Google Colab](https://colab.research.google.com/), [Amazon Web Services (AWS)](https://aws.amazon.com/braket/), etc.

# References &nbsp; <a href="#"><img valign="middle" height="45px" src="https://img.icons8.com/book" width="45" hspace="0px" vspace="0px"></a><a name="References" />

<a name="1"></a> \[1] "Reinforcement Learning: An Introduction." Richard S. Sutton and Andrew G. Barto. [Cambridge, MA: The MIT Press, March 22, 2018, 548 pp.](#).
    
- Complementary material: 
    - [UCL COMPM050/COMPGI13 (2015): RL by David Silver](https://www.davidsilver.uk/teaching/).
    - [UC Berkeley CS285 (Fall 2022): Deep RL by Sergey Levine](http://rail.eecs.berkeley.edu/deeprlcourse/).
    - [Deep RL Bootcamp, Berkeley CA](https://sites.google.com/view/deep-rl-bootcamp/lectures).
    - [John Schulman's PhD Thesis](https://www2.eecs.berkeley.edu/Pubs/TechRpts/2016/EECS-2016-217.html).
    - [Spinning Up in Deep RL](https://spinningup.openai.com/en/latest/).
    - [Williams, R.J. Simple statistical gradient-following algorithms for connectionist reinforcement learning. Mach Learn 8, 229–256 (1992).](https://doi.org/10.1007/BF00992696)

<a name="2"></a> \[2] Deep Learning (Ian J. Goodfellow, Yoshua Bengio and Aaron Courville), MIT Press, 2016.